In [5]:
%AddDeps org.apache.spark spark-sql-kafka-0-10_2.11 2.4.5 
%AddDeps org.apache.kafka kafka-clients 0.10.1.0

Marking org.apache.spark:spark-sql-kafka-0-10_2.11:2.4.5 for download
Obtained 2 files


Waiting for a Spark session to start...

Marking org.apache.kafka:kafka-clients:0.10.1.0 for download
Obtained 2 files


Waiting for a Spark session to start...

In [2]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.catalyst.ScalaReflection
import org.apache.spark.sql.types.StructType
import org.apache.spark.sql.functions.{from_json, window, sum, count, min, max, when, to_json,struct}
import org.apache.spark.sql.streaming.{OutputMode, Trigger}

case class InputJson(event_type: String, category: String,
                       item_id: String, item_price: Long,
                       uid: String, timestamp: Long)


  val bootstrapServer = "spark-master-1:6667"
  val bootstrapServer1 = "10.0.0.5:6667"
  val topicNameIn = "vladimir_vasev"
  val topicNameOut = "vladimir_vasev_lab04b_out"

  val schema = ScalaReflection.schemaFor[InputJson].dataType.asInstanceOf[StructType]

  val dfStream = spark.readStream.
    format("kafka").
    option("kafka.bootstrap.servers", bootstrapServer).
    option("subscribe", topicNameIn).
    load()
 

defined class InputJson
bootstrapServer = spark-master-1:6667
bootstrapServer1 = 10.0.0.5:6667
topicNameIn = vladimir_vasev
topicNameOut = vladimir_vasev_lab04b_out
schema = StructType(StructField(event_type,StringType,true), StructField(category,StringType,true), StructField(item_id,StringType,true), StructField(item_price,LongType,false), StructField(uid,StringType,true), StructField(timestamp,LongType,false))


error: error while loading QualifiedTableName, class file '/usr/hdp/3.1.4.0-315/spark2/jars/spark-catalyst_2.11-2.4.7.jar(org/apache/spark/sql/catalyst/QualifiedTableName.class)' has location not matching its contents: contains class QualifiedTableName
error: error while loading JavaTypeInference, class file '/usr/hdp/3.1.4.0-315/spark2/jars/spark-catalyst_2.11-2.4.7.jar(org/apache/spark/sql/catalyst/JavaTypeInference.class)' has location not matching its contents: contains class JavaTypeInference
error: error while loading FunctionIdentifier, class file '/usr/hdp/3.1.4.0-315/spark2/jars/spark-catalyst_2.11-2.4.7.jar(org/apache/spark/sql/catalyst/FunctionIdentifier.class)' has location not matching its contents: contains class FunctionIdentifier
error: error while loading DefinedByConstructorParams, class file '/usr/hdp/3.1.4.0-315/spark2/jars/spark-catalyst_2.11-2.4.7.jar(org/apache/spark/sql/catalyst/DefinedByConstructorParams.class)' has location not matching its contents: contains 

StructType(StructField(event_type,StringType,true), StructField(category,StringType,true), StructField(item_id,StringType,true), StructField(item_price,LongType,false), StructField(uid,StringType,true), StructField(timestamp,LongType,false))

In [3]:
val dfAgg = dfStream.select(from_json($"value" cast "string", schema) as "record").
    select($"record.*").groupBy(window(($"timestamp" / 1000).
    cast("timestamp"), "1 hour")).agg(
    min("window.start").cast("long") as "start_ts",
    max("window.end").cast("long") as "end_ts",
    sum(when($"event_type" === "buy", $"item_price")) as "revenue",
    count(when($"uid" isNotNull, $"event_type")) as "visitors",
    count(when($"event_type" === "buy", $"event_type")) as "purchases").
    withColumn("aov", $"revenue" / $"purchases").drop("window")

dfAgg = [start_ts: bigint, end_ts: bigint ... 4 more fields]


[start_ts: bigint, end_ts: bigint ... 4 more fields]

In [4]:
 dfAgg.select(to_json(struct("*")) as "value").writeStream.
    format("kafka").
    option("kafka.bootstrap.servers", bootstrapServer).
    option("topic", topicNameOut).
    option("checkpointLocation", "checkpointdir").
    outputMode(OutputMode.Update()).
    trigger(Trigger.ProcessingTime(5000)).
    start().awaitTermination()

Name: java.lang.InterruptedException
Message: null
StackTrace:   at java.util.concurrent.locks.AbstractQueuedSynchronizer.doAcquireSharedInterruptibly(AbstractQueuedSynchronizer.java:998)
  at java.util.concurrent.locks.AbstractQueuedSynchronizer.acquireSharedInterruptibly(AbstractQueuedSynchronizer.java:1304)
  at java.util.concurrent.CountDownLatch.await(CountDownLatch.java:231)
  at org.apache.spark.sql.execution.streaming.StreamExecution.awaitTermination(StreamExecution.scala:469)
  at org.apache.spark.sql.execution.streaming.StreamingQueryWrapper.awaitTermination(StreamingQueryWrapper.scala:53)

In [5]:
spark.stop

lastException: Throwable = null


Waiting for a Spark session to start...